# Check implementation

In [21]:
import bw2data as bd
import bw_processing as bwp
from collections import defaultdict
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from fs.zipfs import ZipFS
from scipy.stats import lognorm

import sys
sys.path.append("/Users/akim/PycharmProjects/akula")
from akula.markets import DATA_DIR, get_dirichlet_scales, select_contributing_exchanges

bd.projects.set_current('GSA for archetypes')

fp_implicit_markets = DATA_DIR / "xgboost" / "implicit-markets-81.zip"
dp = bwp.load_datapackage(ZipFS(fp_implicit_markets))

In [22]:
indices = dp.get_resource('implicit-markets.indices')[0]
data = dp.get_resource('implicit-markets.data')[0]
# distributions = dp.get_resource('implicit markets.distributions')[0]

In [24]:
found = defaultdict(list)
data_act = defaultdict(list)
for col in list(set(indices['col'])):
    rows = indices[indices['col']==col]['row']
    act = bd.get_activity(int(col))
    for exc in act.exchanges():
        if exc.input.id in rows:
            found[act].append(exc)
            row_col = np.array((exc.input.id, col), dtype=indices.dtype)
            where = np.where(indices == row_col)[0][0]
            data_act[act].append(data[where])

In [25]:
write_figs = Path("implicit_markets")
write_figs.mkdir(exist_ok=True, parents=True)

dist_ = {}
num_bins = 100
count = 0
fit_var = True
based_on_contr = False
use_th = False
scaling_factors = get_dirichlet_scales(found, fit_var, based_on_contr, use_th)
  
for ng, current in found.items():
    
    x = np.array([exc['amount'] for exc in current])
    amounts = x.copy()
    amounts_exchanges_dict = {amounts[i]: current[i] for i in range(len(amounts))}
    _, scores = select_contributing_exchanges(amounts_exchanges_dict, use_th, return_scores=True)
    scores_str = [f"{score:4.2e} kg CO2e" for score in scores.values()]
    
    rows=len(current)
    showlegend = True
    x = np.array([exc['amount'] for exc in current])
    alpha = x.copy()
    alpha_exc_dict = {alpha[i]: current[i] for i in range(len(alpha))}

    scaling_factors_str = f"SF={scaling_factors[count]:5.3f}"
    fig = make_subplots(
        rows=rows, 
        cols=1,
        subplot_titles=[scaling_factors_str]
#         subplot_titles = scores_str,
    )

    for i,exc in enumerate(current):
        Y = data_act[ng][i]
        bins_ = np.linspace(min(Y), max(Y), num_bins+1, endpoint=True)
        Y_samples, _ = np.histogram(Y, bins=bins_, density=True)
        # Given distribution
        assert exc['uncertainty type']==2
        loc = exc['loc']
        scale = exc['scale']  
        midbins = (bins_[1:]+bins_[:-1])/2
        Y_distr = lognorm.pdf(midbins, s=scale, scale=np.exp(loc))
        distance = np.sqrt(sum(Y_distr-Y_samples)**2)/max(Y_distr)

        fig.add_trace(
            go.Scatter(
                x = midbins,
                y = Y_samples,
                line_color = 'blue',
                name='Dirichlet samples',
                showlegend=showlegend,
            ),
            row=i+1,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x = midbins,
                y = Y_distr,
                line_color = 'red',
                name='Defined distribution',
                showlegend=showlegend,
            ),
            row=i+1,
            col=1,
        )
        showlegend=False
        fig.update_yaxes(
            title_text=f"{scores[exc.input]:4.2e} kCO2e",
            row=i+1,
            col=1,
        )
    fig.update_layout(
        width=300,
        height=250*rows,
        legend=dict(
            yanchor="top",
            y=-0.2,
            xanchor="left",
            x=0.01,
            orientation='h',
        )
    )
#     fig.add_annotation(x=0, y=1.2,
#             text=scaling_factors_str,
#             showarrow=False,
#             arrowhead=1, font_size=16,        
#             xref="paper", yref="paper",)
    fig.write_html(write_figs / "{}_{}_{}.html".format(count, ng['name'][:20], ng['location'][:3]))
#     fig.show()
#     break
    count += 1

KeyError: 0

# Plot implicit markets for paper 3

In [26]:
names1 = [
    r'$\text{heat and power,}$',
    r'$\text{electricity production,}$',
    r'$\text{electricity production,}$',
    r'$\text{heat and power,}$',
    r'$\text{electricity production,}$',
]
names2 = [
    r'$\text{hard coal, RU}$',
    r'$\text{lignite, BA}$',
    r'$\text{hard coal, ME}$',
    r'$\text{lignite, RU}$',
    r'$\text{lignite, ME}$',
]

In [38]:
color_gray_hex = "#b2bcc0"
color_darkgray_hex = "#485063"
color_black_hex = "#212931"
color_pink_rgb = "rgb(148, 52, 110)"
color_blue_rgb = "rgb(29,105,150)"
color_orange_rgb = "rgb(217,95,2)"
color_red_hex = "#ff2c54"
opacity=0.6
num_bins = 60

ng = list(found)[2]
current = found[ng]

cols = 5
fig = make_subplots(
    rows=1, 
    cols=cols,
    horizontal_spacing=0.05,
    shared_yaxes=False,
#     subplot_titles=names2,
)
showlegend = True
for i,exc in enumerate(current):
    Y = data_act[ng][i]
    bins_ = np.linspace(min(Y), max(Y), num_bins+1, endpoint=True)
    Y_samples, _ = np.histogram(Y, bins=bins_, density=True)
    # Given distribution
    assert exc['uncertainty type']==2
    loc = exc['loc']
    scale = exc['scale']  
    midbins = (bins_[1:]+bins_[:-1])/2
    Y_distr = lognorm.pdf(midbins, s=scale, scale=np.exp(loc))
    distance = np.sqrt(sum(Y_distr-Y_samples)**2)/max(Y_distr)

    fig.add_trace(
        go.Scatter(
            x = midbins,
            y = Y_samples,
            line_color = color_darkgray_hex,
            name=r"$\text{Dirichlet samples}$",
            showlegend=showlegend,
            opacity=opacity,
            line=dict(color=color_blue_rgb, width=1, shape="hvh"),
            fill="tozeroy",
        ),
        row=1,
        col=i+1,
    )
    fig.add_trace(
        go.Scatter(
            x = midbins,
            y = Y_distr,
            line_color = color_red_hex,
            name=r"$\text{Defined lognormal distributions}$",
            showlegend=showlegend,
        ),
        row=1,
        col=i+1,
    )
    showlegend=False
    
fig.update_xaxes(
    title_text=r"$\text{Production volume share}$",
    showgrid=True,
    gridwidth=1,
    gridcolor=color_gray_hex,
    zeroline=True,
    zerolinewidth=1,
    zerolinecolor=color_black_hex,
    showline=True,
    linewidth=1,
    linecolor=color_gray_hex,
)

fig.update_yaxes(title_text=r"$\text{Frequency}$", col=1)
fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor=color_gray_hex,
    zeroline=True,
    zerolinewidth=1,
    zerolinecolor=color_black_hex,
    showline=True,
    linewidth=1,
    linecolor=color_gray_hex,
)

xpos = [0.08, 0.29, 0.5, 0.71, 0.92]
for i in range(cols):
    fig.add_annotation(
        {
            'font': {'size': 14},
            'showarrow': False,
            'text': names1[i],
            'x': xpos[i]-0.08,
            'xanchor': 'left',
            'xref': 'paper',
            'y': 1.2,
            'yanchor': 'bottom',
            'yref': 'paper'
        }
    )
    fig.add_annotation(
        {
            'font': {'size': 14},
            'showarrow': False,
            'text': names2[i],
            'x': xpos[i]-0.08,
            'xanchor': 'left',
            'xref': 'paper',
            'y': 1.05,
            'yanchor': 'bottom',
            'yref': 'paper'
        }
    )
    
fig.update_layout(
    width=220*cols,
    height=250,
    legend=dict(
        yanchor="middle",
        y=-0.7,
        xanchor="center",
        x=0.5,
        orientation='h',
        font=dict(size=13),
        bordercolor=color_darkgray_hex,
        borderwidth=1,
    ),
    margin=dict(t=40, b=10, l=10, r=0),
    paper_bgcolor="rgba(255,255,255,1)",
    plot_bgcolor="rgba(255,255,255,1)",
)
fig.write_image(write_figs / "impicit_market_samples.eps")

# fig.write_html(write_figs / "{}_{}_{}.html".format(count, ng['name'][:20], ng['location'][:3]))
# count += 1

# Plot regression figures for Dirichlet scales for paper 3

In [ ]:
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from gsa_framework.utils import read_pickle
from pathlib import Path
import sys
sys.path.append("/Users/akim/PycharmProjects/akula")
from akula.markets import (
    get_dirichlet_scales, 
    DATA_DIR,
#     MINIMUM_DIRICHLET_SCALE,
#     MAXIMUM_DIRICHLET_SCALE,
)
from akula.utils import update_fig_axes

write_figs = Path("/Users/akim/PycharmProjects/akula/dev/write_files/paper3")

In [ ]:
# ei_name = "ecoinvent 3.8 cutoff"
# found = find_uncertain_implicit_markets(ei_name)
# markets = find_markets(ei_name)
# write_pickle(found, "implicit_markets.pickle")
# write_pickle(markets, "normal_markets.pickle")
ims = read_pickle(DATA_DIR / "implicit-markets.pickle")
gms = read_pickle(DATA_DIR / "generic-markets.pickle")

In [ ]:
LMEAN_ID, NEXC_ID = 0, 1

def get_market_features(markets):
    from bw2analyzer.econ import gini_coefficient
    X = []
    for i, act in enumerate(markets.keys()):
        exchanges = markets[act]
        amounts = np.array([exc['amount'] for exc in exchanges])
        mean = np.mean(amounts)
        lmean = np.mean(amounts[amounts>=mean])
        n_excs = len(exchanges)
        X.append([lmean, n_excs])
    X = np.array(X)
    X_ = 1/X[:,LMEAN_ID]**3
    return X_.reshape((-1,1)), X

Xtrain, Xtrain_base = get_market_features(ims)
ytrain = get_dirichlet_scales(ims, True, True)

In [ ]:
Xtest, Xtest_base = get_market_features(gms)
reg = LinearRegression(fit_intercept=True).fit(Xtrain, ytrain)
ytest = Xtest * reg.coef_
ytest = ytest.flatten()
# ytest[ytest<MINIMUM_DIRICHLET_SCALE] = MINIMUM_DIRICHLET_SCALE
# ytest[ytest>8000] = 8000

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=Xtrain_base[:,0],
        y=ytrain,
        mode="markers",
        marker = dict(
            size=Xtrain_base[:,1]/1.6,
            showscale=False,
            color=Xtrain_base[:,1],
            colorbar_title="# exchanges",
            colorscale="deep",
            line=dict(width=1, color='DarkSlateGrey'),
            opacity=0.8,
        ),
    ),
)
fig = update_fig_axes(fig)
fig.update_xaxes(
    title_text=r"$\text{Mean of above-average amounts}$",
    range=(-0.05,1.05),
    title_standoff=6,
)
fig.update_yaxes(
    title_text=r"$\text{Dirichlet scales}$",
    range=(-800,8100),
    title_standoff=6,
)
fig.update_layout(
    width=600,
    height=200,
    margin=dict(t=40, b=10, l=10, r=10),
    title_text = r"$\text{(i) Implicit markets [training]}$",
)
# fig.write_image(write_figs / "dirichlet_train.pdf")
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.zeros(6),
        y=[5,9.1,13.5,18.2,22.9,28],
        mode="markers",
        marker = dict(
            size=np.array([5,10,15,20,25,30])/1.6,
            showscale=False,
            color=[5,10,15,20,25,30],
            colorbar_title="# exchanges",
            colorscale="deep",
            line=dict(width=1, color='DarkSlateGrey'),
            opacity=0.8,
        ),
    ),
)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.add_annotation(x=0.2, y=34, yanchor="middle", text="# exchanges", showarrow=False)
fig.add_annotation(x=0.5, y=5,  yanchor="middle", text="5", showarrow=False)
fig.add_annotation(x=0.5, y=9.5,  yanchor="middle", text="10", showarrow=False)
fig.add_annotation(x=0.5, y=14, yanchor="middle", text="15", showarrow=False)
fig.add_annotation(x=0.5, y=18.5, yanchor="middle", text="20", showarrow=False)
fig.add_annotation(x=0.5, y=23, yanchor="middle", text="25", showarrow=False)
fig.add_annotation(x=0.5, y=27.5, yanchor="middle", text="30", showarrow=False)
                 
fig.update_layout(
    width=80,
    height=140,
    paper_bgcolor="rgba(255,255,255,1)",
    plot_bgcolor="rgba(255,255,255,1)",
    margin=dict(t=0, b=0, l=0, r=0),
)
# fig.write_image(write_figs / "dirichlet_legend_v.pdf")

In [ ]:
fig = go.Figure()
color_darkgray_hex = "#485063"

fig.add_trace(
    go.Scatter(
        y=np.zeros(6),
        x=[5,9.1,13.5,18.2,22.9,28],
        mode="markers",
        marker = dict(
            size=np.array([5,10,15,20,25,30])[-1::-1]/1.6,
            showscale=False,
            color=[5,10,15,20,25,30][-1::-1],
            colorbar_title="# exchanges",
            colorscale="deep",
            line=dict(width=1, color='DarkSlateGrey'),
            opacity=0.8,
        ),
        showlegend=False
    ),
)

fig.add_trace(
    go.Scatter(
        y=[1.5,1.5],
        x=[3,8],
        mode="lines",
        marker_color = color_red_hex,
        showlegend=False
    ),
)

fig.add_annotation(y= 0.5, x=10, yanchor="middle", text="# exchanges", showarrow=False)
fig.add_annotation(y=-0.5, x=5,  yanchor="middle", text="30", showarrow=False)
fig.add_annotation(y=-0.5, x=9.5,  yanchor="middle", text="25", showarrow=False)
fig.add_annotation(y=-0.5, x=14, yanchor="middle", text="20", showarrow=False)
fig.add_annotation(y=-0.5, x=18.5, yanchor="middle", text="15", showarrow=False)
fig.add_annotation(y=-0.5, x=23, yanchor="middle", text="10", showarrow=False)
fig.add_annotation(y=-0.5, x=27.5, yanchor="middle", text='5', showarrow=False)
fig.add_annotation(y=1.5, x=18, yanchor="middle", text=r'$\text{Regression line}$', showarrow=False)
                 
fig.update_xaxes(showticklabels=False, mirror=True, showline=True, linewidth=1, linecolor=color_darkgray_hex,)
fig.update_yaxes(showticklabels=False, mirror=True, showline=True, linewidth=1, linecolor=color_darkgray_hex,)

fig.update_layout(
    width=200,
    height=140,
    paper_bgcolor="rgba(255,255,255,1)",
    plot_bgcolor="rgba(255,255,255,1)",
    margin=dict(t=40, b=10, l=10, r=10),
    title_text = r"$\text{Legend}$"
)
fig.write_image(write_figs / "dirichlet_legend_h.pdf")

In [ ]:
color_red_hex = "#ff2c54"
fig = go.Figure()
xline = np.linspace(min(Xtrain[:,0]),max(Xtrain[:,0]),100)
fig.add_trace(
    go.Scatter(
        x=Xtrain[:,0],
        y=ytrain,
        mode='markers',
        marker = dict(
            size=Xtrain_base[:,1]/1.6,
#             size = np.ones(len(ims))*8,
            showscale=False,
            color=Xtrain_base[:,1],
            colorbar_title="# exchanges",
            colorscale="deep",
            line=dict(width=1, color='DarkSlateGrey'),
            opacity=0.8,
        ),
        name='Transformed data',
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter(
        x=xline,
        y=reg.coef_[0]*xline + reg.intercept_,
        marker_color=color_red_hex,
        name='Linear regression',
        showlegend=False,
    )
)
fig = update_fig_axes(fig)
fig.update_xaxes(
    title_text=r"$\text{(Mean of above-average amounts)}^{-3}$",
    title_standoff=6,
)
fig.update_yaxes(
    title_text=r"$\text{Dirichlet scales}$",
    title_standoff=6,
)
fig.update_layout(
    width=450,#300,
    height=200,#220,
    margin=dict(t=40, b=10, l=10, r=10),
    title_text = r"$\text{(ii) Linear regression}$",
)
fig.write_image(write_figs / "dirichlet_train_transformed.pdf")

In [ ]:
fig.update_xaxes(
    range=(-40, 550),
)
fig.update_yaxes(
    range=(-80, 1100),
)

fig.update_layout(
    width=450,#300
    height=170,
    margin=dict(t=10, b=10, l=10, r=10),
    title_text="",
)
fig.write_image(write_figs / "dirichlet_train_transformed_zoomin.pdf")

In [ ]:
xtest = Xtest_base[:,0]
mask = np.logical_and(Xtest_base[:,1]<=max(Xtrain_base[:,1]), (ytest < 8000))
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=xtest[mask],
        y=ytest[mask],
        mode='markers',
        marker = dict(
            size=Xtest_base[:,1][mask]/1.6,
            showscale=False,
            color=Xtest_base[:,1][mask],
            colorbar_title="# exchanges",
            colorscale="deep",
            line=dict(width=1, color='DarkSlateGrey'),
            opacity=0.8,
        ),
    )
)
fig = update_fig_axes(fig)
fig.update_xaxes(
    title_text=r"$\text{Mean of above-average amounts}$",
    range=(-0.05,1.05),
    title_standoff=6,
)
fig.update_yaxes(
    title_text=r"$\text{Dirichlet scales}$",
    range=(-400,3100),
    title_standoff=6,
)
fig.update_layout(
    width=600,
    height=200,
    margin=dict(t=40, b=10, l=10, r=10),
    title_text = r"$\text{(iii) Generic markets [prediction]}$",
)
fig.write_image(write_figs / "dirichlet_prediction.pdf")